In [ ]:
from sqlmodel import Field, Session, SQLModel, create_engine, select
from sqlalchemy.orm import declarative_base, sessionmaker

from pathlib import Path
import json
import sqlite3

In [ ]:
import pandas as pd

In [ ]:
parsed_folder = Path('/Users/jameslittiebrant/Data/crs_reports/parsed')
run_time = "20250723_1319"
section_data_path = parsed_folder.joinpath(f"{run_time}_sections.json")
chunk_data_path = parsed_folder.joinpath(f"{run_time}_chunks.json")
metadata_path = parsed_folder.joinpath(f"{run_time}_files.json")

In [ ]:
database_location = Path('/Users/jameslittiebrant/Data/Mycroft/databases')

In [ ]:
db_path = database_location.joinpath('documents.sqlite').absolute()
# engine = sqlite3.connect(db_path)
engine = create_engine(f'sqlite:///{db_path}')

from sqlalchemy import (
    inspect
)
Base = declarative_base()

inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

In [ ]:
# class DocumentDBModel(Base):
#     __tablename__ = "documents"
#     pkey = Column(Integer, primary_key=True, autoincrement=True)
#     id = Column(String)
#     type = Column(String)
#     number = Column(String)
#     active = Column(Boolean)
#     source = Column(String)
#     topics = Column(String)
#     version_id = Column(String)
#     date = Column(String)
#     retrieved_date = Column(String)
#     title = Column(String)


# from sqlmodel import Field, Session, SQLModel, create_engine, select
# from typing import Optional
#
# class DocumentDBModel(SQLModel, table=True):
#     __tablename__ = "documents"
#     p_key: Optional[int] = Field(default=None, primary_key=True)
#     id: str #| None = Field(default=None, primary_key=True)
#     type: str
#     number: str
#     active: bool
#     source: str
#     topics: int
#     version_id: int
#     date: str
#     retrieved_date: str
#     title: str


In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
from datamodels import *

SQLModel.metadata.create_all(engine)

Base.metadata.create_all(engine)

In [ ]:
section_data = pd.read_json(section_data_path)

In [ ]:
section_data.head()

In [ ]:
section_data_dict = section_data.to_dict(orient='records')

In [ ]:
session.bulk_insert_mappings(SectionDBModel, section_data_dict)

In [ ]:
# section_data.to_sql('sections', engine, if_exists='replace', index=False)

In [ ]:
chunk_data = pd.read_json(chunk_data_path)

In [ ]:
chunk_data.head()

In [ ]:
chunk_data['element_ids'] = chunk_data['element_ids'].str.join(',')

In [ ]:
chunk_data.dtypes

In [ ]:
chunk_data_dict = chunk_data.to_dict(orient='records')

In [ ]:
session.bulk_insert_mappings(ChunkDBModel, chunk_data_dict)

In [ ]:
metadata_data = pd.read_json(metadata_path, convert_dates=False)

In [ ]:
metadata_data.head()

In [ ]:
metadata_data['topics'] = metadata_data['topics'].str.join('|')

In [ ]:
metadata_data.dtypes

In [ ]:
metadata_data_dict = metadata_data.to_dict(orient='records')

In [ ]:
session.bulk_insert_mappings(DocumentDBModel, metadata_data_dict)

In [ ]:
session.commit()

In [ ]:
session.close()

In [ ]:
engine.dispose()